In [11]:
from itertools import combinations
from itertools import combinations_with_replacement

example = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

manual = []
manual2 = []
for line in example.split("\n"):
    diagram = [x for x in line.split("]")[0][1:]]
    buttons = [tuple(int(y) for y in x[1:-1].split(",")) for x in line.split("]")[1].split("{")[0].strip().split(" ")]
    machine = [diagram, buttons]
    manual.append(machine)
    joltage = [int(x) for x in line.split("]")[1].split("{")[1][:-1].strip().split(",")]
    manual2.append([buttons, joltage])

manual2

[[[(3,), (1, 3), (2,), (2, 3), (0, 2), (0, 1)], [3, 5, 4, 7]],
 [[(0, 2, 3, 4), (2, 3), (0, 4), (0, 1, 2), (1, 2, 3, 4)], [7, 5, 12, 7, 2]],
 [[(0, 1, 2, 3, 4), (0, 3, 4), (0, 1, 2, 4, 5), (1, 2)],
  [10, 11, 11, 5, 10, 5]]]

In [9]:
with open("input.txt") as f:
    input_text = f.read().strip()

manual = []
manual2 = []
for line in input_text.split("\n"):
    diagram = [x for x in line.split("]")[0][1:]]
    buttons = [tuple(int(y) for y in x[1:-1].split(",")) for x in line.split("]")[1].split("{")[0].strip().split(" ")]
    machine = [diagram, buttons]
    manual.append(machine)
    joltage = [int(x) for x in line.split("]")[1].split("{")[1][:-1].strip().split(",")]
    manual2.append([buttons, joltage])

manual2

[[[(0, 3), (0, 2, 3), (1, 3), (0, 1, 3)], [117, 15, 2, 128]],
 [[(2, 3), (5, 6), (0, 1, 3, 4, 5), (0, 1, 2, 6), (2, 3, 4, 5, 6), (0, 1, 5)],
  [12, 12, 13, 13, 2, 31, 19]],
 [[(0, 1, 3, 7, 8),
   (0, 1, 2, 3, 4, 6, 7, 9),
   (0, 1, 3, 5, 6, 8, 9),
   (3, 8),
   (1, 6, 7, 9),
   (2, 8),
   (1, 3, 5),
   (0, 6, 9),
   (4, 8),
   (0, 1, 3, 6, 7, 8, 9),
   (0, 2, 5, 6, 8)],
  [51, 33, 24, 38, 16, 22, 58, 25, 54, 44]],
 [[(0, 1, 2, 3), (0, 2, 3)], [9, 1, 9, 9]],
 [[(0, 1, 4, 5), (0, 1, 3, 5), (0, 1, 4), (2, 3), (0, 1, 3, 4, 5), (1, 3)],
  [43, 59, 16, 39, 38, 25]],
 [[(2, 6, 8),
   (2, 5, 6, 7, 8, 9),
   (0, 1, 6, 7, 8, 9),
   (0, 1, 2, 5, 6, 7, 8, 9),
   (3, 4),
   (0, 1, 3, 4, 5, 6, 7, 9),
   (3, 4, 7, 8, 9),
   (1, 2, 4)],
  [24, 40, 67, 26, 42, 34, 62, 64, 81, 64]],
 [[(1, 4, 5, 6, 7),
   (0, 2, 3, 4, 5, 6, 8),
   (1, 2, 4, 7, 8),
   (0, 2, 5, 6, 8),
   (0, 1, 5, 7),
   (1, 3, 6),
   (3, 5),
   (0, 1, 2, 6),
   (2, 3, 7)],
  [44, 44, 44, 36, 12, 42, 46, 36, 27]],
 [[(2, 3), (1, 4), (0, 

In [3]:
def fewest_button_presses(goal_diagram, buttons):
    for n_combinations in range(1, len(buttons) + 1):
        for combination in list(combinations(buttons, n_combinations)):
            start_diagram = ["." for x in range(len(goal_diagram))]
            for button in combination:
                for num in button:
                    if start_diagram[num] == ".":
                        start_diagram[num] = "#"
                    elif start_diagram[num] == "#":
                        start_diagram[num] = "."
            if start_diagram == goal_diagram:
                return n_combinations
                
    
button_presses = []
for goal_diagram, buttons in manual:
    button_presses.append(fewest_button_presses(goal_diagram, buttons))

sum(button_presses)

461

In [4]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 11.2 MB/s  0:00:03m0:00:0100:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
from z3 import Int, Optimize, Sum, sat

with open("input.txt") as f:
    input_text = f.read().strip()


# ---------- PARSING ----------

def parse_machines(text):
    machines = []  # list of (buttons, target_joltage)

    for line in text.splitlines():
        # Split diagram / buttons+target
        before, after = line.split("]", 1)   # before: "[.##."  after: " (3) (1,3) ... {3,5,4,7}"

        # Extract button part = between ']' and '{'
        buttons_part = after.split("{")[0].strip()   # "(3) (1,3) (2) (2,3) (0,2) (0,1)"

        # Extract target joltage between '{' and '}'
        joltage_str = after.split("{")[1].rstrip("}")
        target = [int(x) for x in joltage_str.split(",")]

        # Parse buttons: "(1,3)" -> (1, 3)
        if buttons_part:
            button_specs = buttons_part.split()
            buttons = [
                tuple(int(idx) for idx in spec.strip()[1:-1].split(","))
                for spec in button_specs
            ]
        else:
            buttons = []

        machines.append((buttons, target))

    return machines


machines = parse_machines(input_text)


# ---------- Z3 MODEL FOR A SINGLE MACHINE ----------

def fewest_button_presses_z3(buttons, target):
    """
    buttons: list[tuple[int]]   each tuple is the list of counters this button increments
    target: list[int]           desired final value for each counter

    Returns: minimal total number of button presses to reach exactly `target`,
             or raises if no solution.
    """
    n_buttons = len(buttons)
    n_counters = len(target)

    # One non-negative integer variable per button: how many times we press it.
    x = [Int(f"x_{i}") for i in range(n_buttons)]

    opt = Optimize()

    # Each x_i >= 0
    for var in x:
        opt.add(var >= 0)

    # For each counter j, sum over all buttons that affect it:
    #   sum(x_i * 1_if_button_i_hits_j) == target[j]
    for j in range(n_counters):
        contribs = []
        for i, btn in enumerate(buttons):
            hits = 1 if j in btn else 0
            if hits:
                contribs.append(x[i] * hits)
        # If no button ever touches this counter, the only valid target is 0
        if contribs:
            opt.add(Sum(contribs) == target[j])
        else:
            opt.add(target[j] == 0)

    # Minimize total button presses
    total_presses = Sum(x)
    opt.minimize(total_presses)

    # Solve
    if opt.check() != sat:
        raise RuntimeError("No solution found")

    model = opt.model()
    return model.eval(total_presses).as_long()


# ---------- APPLY TO ALL MACHINES ----------

total = 0
for buttons, joltage in machines:
    total += fewest_button_presses_z3(buttons, joltage)

print("Result 2:", total)


Result 2: 16386
